# <font style="color:blue">Mask / No Mask Detector - YOLO v4 - Windows</font>
In this notebook we will train a custom object detector using YOLO v4,  Specially a face mask detector.

Just for your consideration, we will be creating a folder named YOLO. In the parent directory is located this notebook, but all other files are inside the YOLO folder. 

On the other hand, we will be working with Visual Studio 2019.  So you will have to make this minor change in cmake to work. Remember to fix the architecture if you are not using Visual Studio 2019 (default in other is 32-bits, we will be working on 64-bits).

Pre-requisites for this to work faster is to install CUDA Toolkit and cudnn libraries from the NVIDIA website.

The results of this windows machine where over a NVIDIA GeForce 1660Ti.

**NOTES**:
- *POWERSHELL DOWNLOADING PRETRAINED WEIGHTS TO TEST THE COMPILATION OF DARKNET IS REALLY SLOW, YOU WILL PREFFER TO DO IT MANUALLY*
- *THE OUTPUTS OF THE TRAINING ARE CLEARED, WE SHARED TO YOU THE RESULTS AND THE PRE-TRAINED WEIGHT, SEE THE README.MD*

### <font color="blue">0. Importing and Helper Functions </font>

In this section we will make use of our helper functions that will ease the use over the entire notebook.

Just some imports:
- os : path handling
- cv2: image processing
- IPython: for audio play
- datetime: for getting the timestamp
- random: for shuflying data
- subprocess: for calling functions of the system

In [ ]:
import random
import os
import subprocess
import sys
import cv2
import matplotlib.pyplot as plt
from IPython.display import Audio
from datetime import datetime
%matplotlib inline

This is where a nice soundwave is located on the machine for just tell us when the training ended or when processing YOLO video when it ends to patch the video with the predictions.

Trainings vary from 2 hrs to 30 hrs so is good if you are near to hear that it finished.

In [ ]:
sound_file = 'C:\Windows\Media\Ring10.wav'

The utility function below will help us to display the test image and its predicted labels, also will save the picture on the results folder

In [ ]:
# Utility function to display the output

def display_output(imagePath):
    src = cv2.imread(imagePath,1)
    output = cv2.imread("predictions.jpg")
    
    plt.figure(figsize=[20,8])
    plt.subplot(121)
    plt.imshow(src[:,:,::-1])
    plt.title("Original Image")
    plt.subplot(122)
    plt.imshow(output[:,:,::-1])
    plt.title("Predictions")
    
    file_name = imagePath.split('\\')
    file_name_parts = file_name[2].split('.')
    name_only = file_name_parts[0]
    now = datetime.now() # current date and time
    build = now.strftime("%m-%d-%Y_%H_%M_%S")
    file_save_path = os.path.join(file_name[0], 'result', 'yolov4')
    save_name = os.path.join(file_save_path, name_only + '-yolov4--' + build +'.png')
    plt.savefig(save_name)    
    
    plt.show()    

This class will help us to read a file and modify their lines in a easy way.  An example of usage is in comments below.  On the other hand you will use it on the entire document.

In [ ]:
# Utility class to modify files

# Get a file handler
# mkf = FileMod(doc='.\Makefile')  

# Load the file and view the line numbers
# mkf.load(doc='.\Makefile')

# make a dictionary to modify the file
# mkf_dict = dict({
#                  0:'GPU=1', 
#                  2: 'CUDNN_HALF=1'
#                })
# mkf.mod(doc='.\Makefile', mod_lines=mkf_dict)

# view the print out without line numbers (optional)
# mkf.view()   # view 20 lines
# ... or ...
# mkf.view(10) # view 10 lines

class FileMod():
    def __init__(self, doc=None, *args, **kwargs):
        super(FileMod, self).__init__()
        
        self.mode  = 'r'  # default = read
        self.file  = doc # path of file
        self.dict  = {}   # dict to modify
        self.lines = []   #lines of file

    def load(self, doc=None):
        try:
            if doc==None:
                doc=self.file
            with open(doc, self.mode) as file:
                # read a list of lines into data
                data = file.readlines()
            self.lines = data
            for i, string in enumerate(self.lines):
                print(f"{i}:{string}")
        except Exception as e:
            print(e)

    def mod(self, mod_lines=None):
        # and write everything back
        if mod_lines == None:
            return
        doc = self.file
        self.mode = 'w'
        for lines, data in mod_lines.items():
            string = data + '\n'
            self.lines[lines] = string
            print(f"mod {lines} as {string}")
        with open(doc, 'w') as file:
            file.writelines(self.lines)
        self.mode = 'r'

    def view(self, lines=None):
        if lines == None:
            lines = 20
        doc = self.file
        with open(doc, self.mode) as file:
            # read a list of lines into data
            data = file.readlines()
        for line_num, line in enumerate(data):
            if line_num > lines - 1:
                break
            print(f"{line}\n")

In each step I am doing a dummy *ls* for seein in which folder we are

In [ ]:
%ls

We will then create a folder named *YOLO* and will start working from here

In [ ]:
%mkdir YOLO
%cd YOLO

### <font color="blue">1.  Download Alexey AB YOLO repository to our google drive *yolov4* folder</font>
We now will download the well maintained repository of a YOLO expert, because Joseph Redmon will not continue working on YOLO and because this repo work very well on Windows.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

### <font color="blue">2.  Compile Darknet</font>
Next, some strings on the Makefile needs to be modified before we build darknet from the source.

In [ ]:
%ls

In [ ]:
%cd darknet\build

In [ ]:
mkf = FileMod('..\Makefile')
mkf.load()

In [ ]:
mkfdict = dict({
                0:'GPU=1',
                1:'CUDNN=1',
                3:'OPENCV=1',
                4:'AVX=1'
              })
mkf.mod(mod_lines=mkfdict)

In [ ]:
mkf.view()

In [ ]:
%ls

In [ ]:
print("[INFO] - Building Darknet with cmake")
!cmake -G "Visual Studio 16 2019" ..

In [ ]:
print("[INFO] - Building Darknet Release Mode")
!cmake --build . --target ALL_BUILD
print("[INFO] - End of building darknet!")

In [ ]:
print("[INFO] - Final Steps")
!cmake --build . --target INSTALL
print("[INFO] - You are done for start training or detecting!")

### <font color="blue">3. Move DLLs To The Root Folder of Darknet</font>

Darknet will need phtreadVC2.dll to load correctly so we will be putting it on the root of the darknet folder for simplifying things.

In [ ]:
%cd ..\

In [ ]:
%ls

In [ ]:
!copy 3rdparty\pthreads\bin\* .

### <font color="blue">4. Cloning the Pre-Trained Weights</font>
We will be doing a compilation test with the pre-trained weights of YOLO v4 (just this time).

In [ ]:
%ls

In [ ]:
# We will download the pre-trained weights of yolov4 just this time to test the darknet compilation
# This will take a lot of time, you preferibly will want to do it manually
!powershell -c "(New-Object System.Net.WebClient).DownloadFile('https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights', 'yolov4.weights')"
Audio(sound_file, autoplay=True)

### <font color="blue">5. Test The Compilation</font>

If everything is OK, in predictions.jpg you will get the image with their bounding boxes and classifications of the classes detected by yolo.

In [ ]:
!darknet detect cfg\yolov4.cfg yolov4.weights data\dog.jpg -dont_show > NUL

### <font color="blue">6.  Download the *Mask / No Mask Dataset*</font>
Download the curated dataset of face mask and not facemask to our dataset folder, but first lets create a dataset folder.

Usually i like to be very organized.

In [ ]:
%ls

In [ ]:
%cd ../

Next we will download the dataset to this root folder *YOLO*, decompress it on another folder called *dataset* and finally erase the zip file.

In [ ]:
!powershell -c "(New-Object System.Net.WebClient).DownloadFile('https://www.dropbox.com/s/6gewe947ake1g95/kaggle_and_no-mask_dataset.zip?dl=1', '.\kaggle_and_no-mask_dataset.zip')"
!powershell -c "expand-archive -path 'kaggle_and_no-mask_dataset.zip' -destinationpath 'dataset'"
!powershell -c "remove-item -path 'kaggle_and_no-mask_dataset.zip'"

### <font color="blue">7.  Prepare the Train and Test Files</font>
This code provided creates two files, one for the training images and other for the test images.

The dataset is divided in:
- 80% training set 
- 20% test set

In [ ]:
%ls

In [ ]:
image_dir = "dataset"
f_val = open("DS_test.txt", 'w')
f_train = open("DS_train.txt", 'w')

path, dirs, files = next(os.walk(image_dir))
data_size = len(files)

ind = 0
data_test_size = int(0.2 * data_size)
test_array = random.sample(range(data_size), k=data_test_size)

for f in os.listdir(image_dir):
    if(f.split(".")[-1] == "jpg"):
        ind += 1
        
        if ind in test_array:
            f_val.write(image_dir+'/'+f+'\n')
        else:
            f_train.write(image_dir+'/'+f+'\n')

f_train.close()
f_val.close()

### <font color="blue">8.  Prepare and Upload the configuration files</font>

For train and use yolo we need to prepare the neccesary files, finally we will upload to our yolov4 folder.

#### <font color="blue">8.1. Data Setup</font>
In the file **`yolov4-mask_nomask-setup.data`**(uploaded to github and included here), we provided the correct specification of our paths.

Below is the content of this file:
```
classes = 2
train  = DS_train.txt
valid  = DS_test.txt
names = FacialMaskDetector/yolov4/config/yolov4.names
backup = FacialMaskDetector/yolov4/backup/
```

#### <font color="blue">8.2. Training Config File</font>
We also need to provide the  **`yolov4-mask_nomask-train.cfg`**. Based on the default file of darknet folder cfg/yolov4.cfg

#### <font color="blue">8.2.1 Batch hyper-parameter in YOLOv4</font>
We maintain the batch size and the subdivision batch as the default of the file.  That's 20 images to feed in two subgroups.

For the *test configuration file* we will set these params to 1 because we will feed an image.
```
batch=20
subdivisions=10
```
#### <font color="blue">8.2.2 Subdivisions configuration parameter in YOLOv4</font>
For feeding the GPU correcly we slow the training, this action will ensure our machine will be able to train the detector but you could play with this parameter.

In testing mode this will be equal to 1.

### <font color="blue">8.2.3 Width, Height, Channels</font>
A higher resolution the detection will be better but you will sacrifice the time of training.
```
width=352
height=352
channels=3
```
### <font color="blue">8.2.4 Momentum and Decay</font>
These hyperparameters we will maintain as is.
```
momentum=0.949
decay=0.0005
```

### <font color="blue">8.2.5 Learning Rate, Steps, Scales, Burn In (warm-up)</font>
Again, we will be maintaining these hyperparamters as default.  The learning rate will be changed over the 800th iteration.

```
learning_rate=0.0013
policy=steps
steps=6000
scales=.1,.1
burn_in=600
```


### <font color="blue">8.2.6 Data augmentation</font>
For various lighting conditions and different color ranges (over other color spaces).  Sometimes its good to change the angle because a few objects could be rotated.  A facemask in this case we will maintain at 0 degrees but could be augmented to 3° or something low because we are detecting people.
```
angle=0
saturation = 1.5
exposure = 1.5
hue=.1
```

#### <font color="blue">8.2.7 Number of iterations</font>
Process 2000*n_classes iterations run as recommended.  But we will keep in a higher number to see the performance.
```
max_batches=6000
```

#### <font color="blue">8.2.8 Change filters parameter in conv layers [Important for multiple objects]</font>
Fini in the file the conv. layer before the yolo layer with the equation:

**`filters=( classes + 5 ) * 3`**

Our yolo classes are 2, so filters = 21.

### <font color="blue">8.3. yolov4.names file </font>
Specify the classes as...
- Mask (Class 0), 
- No-Mask (Class 1) 

... class labels.

### <font color="blue">8.4. Improving The Network </font>
We observed that on the AlexeyAB repo recommended change some hyperparameters for improving the training accuracy.  These parameters modified are:
- stride=4, near line 890
- layers=23, near line 894
- added max=200 or above at the end of the file

### <font color="blue">9. Cloning the configuration Repository</font>
This repository is prepared with the desired configuration for yolov3 and yolov4.

In this case we will be using yolo v4.  The structure of the folder is as follows:

```
/FacialMaskDetector
├── results
│   ├── yolov3
│   │       ├── README.md
│   ├── yolov4
│           ├── README.md
├── test
│   ├── test-image1.jpg
│   ├── test-image2.jpg
│   ├── test-image3.jpg
│   ├── test-image4.jpg
│   ├── test-video1.jpg
│   ├── test-video2.jpg
├── yolov3
│       ├── backup
│       │   ├── README.md
│       ├── config
│       │   ├── yolov3-mask_nomask-setup.data
│       │   ├── yolov3-mask_nomask-test.cfg
│       │   ├── yolov3-mask_nomask-train.cfg
│       │   ├── yolov3.names
│       ├── weights
│           ├── README.md
├── yolov4
│       ├── backup
│       │   ├── README.md
│       ├── config
│       │   ├── yolov4-mask_nomask-setup.data
│       │   ├── yolov4-mask_nomask-test.cfg
│       │   ├── yolov4-mask_nomask-train.cfg
│       │   ├── yolov4.names
│       ├── weights
│           ├── README.md
├── README.md
```

This will ease for us the way of training and inference.

In [ ]:
%ls

In [ ]:
!git clone https://github.com/issaiass/FacialMaskDetector.git

We need to download the initial weights

In [ ]:
%cd FacialMaskDetector\yolov4\weights

In [ ]:
!powershell -c "(New-Object System.Net.WebClient).DownloadFile('"https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"', 'yolov4.conv.137')"
Audio(sound_file, autoplay=True)

In [ ]:
%cd ..\..\..\

### <font color="blue">10. Review Hyperparameters</font>

Here we will set the hyperparamters for training the YOLO v4 model.
Below python variables to be the shortcut for fast insertion on the code.

In [ ]:
yolo_setup     = 'FacialMaskDetector\\yolov4\\config\\yolov4-mask_nomask-setup.data'
yolo_train     = 'FacialMaskDetector\\yolov4\\config\\yolov4-mask_nomask-train.cfg'
yolo_test      = 'FacialMaskDetector\\yolov4\\config\\yolov4-mask_nomask-test.cfg' 
yolo_weights      = 'FacialMaskDetector\\yolov4\\weights\\yolov4.conv.137'
yolo_best_weights = 'FacialMaskDetector\\yolov4\\backup\\yolov4-mask_nomask-train_best.weights'


In [ ]:
%ls

In [ ]:
CfgHandler = FileMod(yolo_train)
CfgHandler.load()

Enable if you want to modify something in the file.  If you didn't know what linew we changed, but its obvious, you could check at: 
- https://www.diffnow.com/
- https://text-compare.com/Enable if you want to modify something in the file.  If you didn't know what linew we changed, but its obvious, you could check at https://www.diffnow.com/

In [ ]:
#                                            #  DEFAULT
#config = dict({ 1:'batch=20',                # 64
#                2:'subdivisions=10',         # 8
#                6:'width=320',               # 512
#                7:'height=320',              # 512
#               19:'learning_rate=0.0013',    # 0.0013
#               17:'burn_in=600',             # 1000
#               18:'max_batches=6000',        # 500500
#               20:'steps=4800,5400',         # 400000,450000
#               21:'scales=.1,.1'             # .1,.1
#              890:'stride=4'                 # 2
#              894:'layers=23'                # 54
#              960:'filters=21'               # 255
#              967:'classes=2'                # 80
#              988:'stride=4'                 # 2
#             1048:'filter=21'                # 255
#             1055:'classes=2'                # 80
#             1136:'filters=21'               # 255
#             1143:'classe=2'                 # 80
#             1157:'max=200'                  # <just leav it in blank with ''>
#             })

#CfgHandler.mod('FacialMaskDetector/yolov4/config/yolov4-mask_nomask-train.cfg', config)

In [ ]:
CfgHandler.view(22)

### <font color="blue">11. Start Training </font>
We need to pass the training files for the darknet framework to start the process.

Remember!, we need to specify the complete paths of these files that we talked before:
1. setup file, 
1. config file, 
1. convolutional weights file 

There are a few flags like **`dont_show`** which wont display the graphs and **`map`** - for the mAP calculation over the DS_test.txt file (20% of our data).

In [ ]:
%ls

Below the paths to insert as a variable in the command line as arguments, and to not repeat in the entire notebook

We added an audio sound file, if you are near you will hear a ring tone.

This is the same as:

*!darknet\darknet detector train FacialMaskDetector\yolov4\config\yolov4-mask_nomask-setup.data FacialMaskDetector\yolov4\config\yolov4-mask_nomask-train.cfg FacialMaskDetector\yolov4\weights\yolov4.conv.137 -dont_show -map 2 > train_log.txt*

In [ ]:
# Train!
!darknet\darknet detector train $yolo_setup $yolo_train $yolo_weights -dont_show -map 2 > train_log.txt
Audio(sound_file, autoplay=True)

Our results:
- class_id = 0, name = Mask, ap = 92.92%   	 (TP = 490, FP = 56) 
- class_id = 1, name = No-Mask, ap = 75.62%   	 (TP = 725, FP = 92) 

- for conf_thresh = 0.25, precision = 0.89, recall = 0.75, F1-score = 0.82 
- for conf_thresh = 0.25, TP = 1215, FP = 148, FN = 396, average IoU = 70.59 % 

- IoU threshold = 50 %, used Area-Under-Curve for each unique Recall 
- mean average precision (mAP@0.50) = 0.842709, or 84.27 % 
 

And the training time was about less than 8 hours depending of the max_batches and width x height of the configuration in the yolo training file.

### <font color="blue">12. Make Inferences</font>

First we did a first step that is to copy the data folder of darknet to the root, because inside are the labels to graph the letters and numbers.  If this step is omited you will see a black box and no inference name.

Next we will have a piece of code for displaying and finally are the inferences of images and pre-recorded video.

The pre-recorded video could be downloaded over the root folder, in this case, our YOLO folder (created on Google Drive) 

In [ ]:
%ls

In [ ]:
!mkdir data
!Xcopy /E /I darknet\data\* data

Below we will list the test files

In [ ]:
test_files = [os.path.join('FacialMaskDetector\\test', v) for v in os.listdir('FacialMaskDetector\\test')]

#### <font color="blue">12.1 Scenario 1</font>

Testing only with one man and with facemask.

In [ ]:
!darknet\darknet detector test $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-image1.jpg -thresh .6 2 > NUL

In [ ]:
display_output(test_files[0])

#### <font color="blue">12.2 Scenario 2</font>

Testing with different angles of people wearing mask and other not wearing.

In [ ]:
!darknet\darknet detector test $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-image2.jpg -thresh .6 2 > NUL

In [ ]:
display_output(test_files[1])

#### <font color="blue">12.3 Scenario 3</font>

Testing with a crowd with and without facemask and different scales and variations

In [ ]:
!darknet\darknet detector test $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-image3.jpg -thresh .6 2 > NUL

In [ ]:
display_output(test_files[2])

#### <font color="blue">12.4 Scenario 4</font>

Testing with several people walking trough the plaza.

In [ ]:
!darknet\darknet detector test $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-image4.jpg -thresh .6 2 > NUL

In [ ]:
display_output(test_files[3])

#### <font color="blue">10.5 Scenario 5</font>

Testing over a video of people wearing or not wearing mask, good lighting conditions.  Probably will take between 1.5 to 3 min for generate the video.

In [ ]:
!darknet\darknet detector demo $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-video1.mp4 -thresh .6 2 -out_filename out-vid1.avi -dont_show
Audio(sound_file, autoplay=True)

#### <font color="blue">10.6 Scenario 6</font>

A video of the crowd.  Variable ligth conditions.  The video will be generated between 5 to 7 minutes.

In [ ]:
!darknet\darknet detector demo $yolo_setup $yolo_test $yolo_best_weights FacialMaskDetector\test\test-video2.mp4 -thresh .6 2 -out_filename out-vid2.avi -dont_show
Audio(sound_file, autoplay=True)

Finally, move the two videos to the result folder

In [ ]:
!powershell -c "Move-Item out-vid1.avi FacialMaskDetector\\result\\yolov4\\"
!powershell -c "Move-Item out-vid2.avi FacialMaskDetector\\result\\yolov4\\"

### <font color="blue">13. Conclusion</font>

This concludes the yolo v3 training and inference google colab notebook.

The disadvantage of GPU for large training datasets will leave you to wait the availability of free GPUs on the cloud (Colab/Kaggle).

There are other alternatives like Azure, Watson and Amazon Web Services, but those ones charge you by GPU usage over time.  

For this and other reasons people like to build their "low cost" deep learning rig to prepare their trainers. 

If you like to retrain, you could always start with the final weights and retrain over it, continuing with the last point of training will converge faster to your solution.

